<table align="left">
  <td>
    <a href="https://colab.research.google.com/drive/1d8lmQCL40i0rGj_qMFjj5dUXdQYR_ixe" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

---

# **Licencia**

**Autor**: Juan Francisco Puentes Calvo

**Licencia**: GPL v3 (https://www.gnu.org/licenses/gpl-3.0.html)


# **Reconocimientos**

* Ninguno, por ahora.

---

## Reconocimiento de gestos en las manos

**Esta celda se ejecuta en un Jupyter local, para tener acceso a la webcam.**

In [12]:
!pip -q install ipywidgets jupyter-ui-poll mediapipe

In [ ]:
# prompt: Crea un interfaz que abra la cámara con varios botones. Habrá en total 3 botones, etiquetados como piedra, papel y tijera. Al pulsar cualquiera de estos botones se almacenará una imagen de nombre "<etiqueta><número>.png", en el subdirectorio "manos", donde <etiqueta> es la etiqueta del botón y <número> es un contador. La webcam permanecerá abierta hasta que se pulse "q".

import cv2
import ipywidgets as widgets
from IPython.display import display
import os
from jupyter_ui_poll import ui_events;

# Crea el directorio "manos" si no existe
if not os.path.exists("manos"):
    os.makedirs("manos")

# Contadores para cada gesto
contador_piedra = 0
contador_papel = 0
contador_tijera = 0

# Función para capturar y guardar una imagen
def capturar_imagen(etiqueta):
    global cap, contador_piedra, contador_papel, contador_tijera

    # Incrementa el contador correspondiente
    if etiqueta == "piedra":
        contador_piedra += 1
        nombre_archivo = f"manos/piedra{contador_piedra}.png"
    elif etiqueta == "papel":
        contador_papel += 1
        nombre_archivo = f"manos/papel{contador_papel}.png"
    elif etiqueta == "tijera":
        contador_tijera += 1
        nombre_archivo = f"manos/tijera{contador_tijera}.png"

    # Captura la imagen
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(nombre_archivo, frame)
        print(f"Imagen guardada como {nombre_archivo}")

# Crea los botones
boton_piedra = widgets.Button(description="Piedra")
boton_papel = widgets.Button(description="Papel")
boton_tijera = widgets.Button(description="Tijera")

# Define las funciones para manejar los eventos de los botones
def on_click_piedra(b):
    capturar_imagen("piedra")
def on_click_papel(b):
    capturar_imagen("papel")
def on_click_tijera(b):
    capturar_imagen("tijera")

# Asigna las funciones a los eventos de los botones
boton_piedra.on_click(on_click_piedra)
boton_papel.on_click(on_click_papel)
boton_tijera.on_click(on_click_tijera)

# Muestra los botones
display(boton_piedra, boton_papel, boton_tijera)

# Abre la cámara
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("No se pudo abrir la cámara")
    cap.release()
else:
    print("Cámara abierta correctamente")

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    # Bucle principal
    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imshow("Webcam", frame)
            with ui_events() as poll: poll(10)
        else:
            break
        # Espera la tecla 'q' para salir
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Libera los recursos
    cap.release()
    cv2.destroyAllWindows()


In [23]:
# prompt: En el subdirectoprio "manos" hay imágenes, cada una de una mano. Busca la imagen llamada "papel105.png", aplicale una identificación de gesto usando mediapipe y muestra la imagen con los puntos de referencia.

import mediapipe as mp
import cv2

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Carga la imagen
imagen_path = "manos/papel105.png"
imagen = cv2.imread(imagen_path)

# Inicializa el modelo de detección de manos
with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
    # Procesa la imagen
    resultados = hands.process(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))

    # Dibuja los puntos de referencia si se detectaron manos
    if resultados.multi_hand_landmarks:
        for hand_landmarks in resultados.multi_hand_landmarks:
            mp_drawing.draw_landmarks(imagen, hand_landmarks, mp_hands.HAND_CONNECTIONS)

# Muestra la imagen con los puntos de referencia
cv2.imshow("Imagen con puntos de referencia", imagen)
cv2.waitKey(0)
cv2.destroyAllWindows()


I0000 00:00:1723470600.467029    5710 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723470600.544901   30058 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 4070/PCIe/SSE2
W0000 00:00:1723470600.549030   30045 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723470600.556457   30044 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Los nombres de los ficheros de cada imagen es del tipo "<label><number>.png". Por ejemplo "papel43.png".

Para extraer la etiqueta hay que eliminar la extensión y el número.

In [ ]:
# prompt: En el subdirectoprio "manos" hay imágenes, cada una de una mano. Para cada una de ellas haz una identificación del gesto usando mediapipe, obten los puntos de referencia de la mano, normalízalo respecto al punto que representa la muñeca. Almacena cada uno de estos vectores en un Dataframe. La última columna de cada fila es la etiqueta. No muestres nada.

import mediapipe as mp
import pandas as pd

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Inicializa el DataFrame
df = pd.DataFrame()

# Itera sobre los archivos en el directorio "manos"
for filename in os.listdir("manos"):
    if filename.endswith(".png"):
        # Carga la imagen
        image_path = os.path.join("manos", filename)
        image = cv2.imread(image_path)

        # Procesa la imagen con MediaPipe
        with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Extrae los puntos de referencia de la mano
                    landmarks = []
                    for landmark in hand_landmarks.landmark:
                        landmarks.append([landmark.x, landmark.y, landmark.z])

                    # Normaliza los puntos de referencia respecto a la muñeca (punto 0)
                    wrist = landmarks[0]
                    normalized_landmarks = [[x - wrist[0], y - wrist[1], z - wrist[2]] for x, y, z in landmarks]

                    # Extrae la etiqueta del nombre del archivo
                    label = filename.split('.')[0][:-1]
                    label = label.rstrip('0123456789')

                    # Agrega los puntos de referencia normalizados y la etiqueta al DataFrame
                    row = normalized_landmarks[1:]  # Excluye la muñeca
                    row.append(label)
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

# Nombra las columnas del DataFrame
num_landmarks = len(df.columns) - 1
column_names = [f"Landmark_{i}" for i in range(num_landmarks)]
column_names.append("Label")
df.columns = column_names


In [22]:
# prompt: Guarda el dataframe 'df' en un fichero llamado 'manos.csv'.

df.to_csv('manos.csv', index=False)


### Dado un dataframe con vectres de características anotados, hacer un clustering